# Multi Layer Perceptron

Author: bedezub

In [29]:
import numpy as np
from math import exp
import bpnn

### Step 1: Initialization

Set all the weights and threshold levels of the network

Modified version of input, python 0-indexing so we start with Neuron 0 hehe

Neuron 0 and Neuron 1 and so on...

In [2]:
learning_rate = 0.1

In [3]:
x_inputs = np.array([[1,1]]) 

Weight at hidden layer

Row      = number of neuron at hidden layer. Contoh, index 0 is first row and index 1 is second row  
Column   = value of neutron at input layer aka value yang masuk dalam each neuron

In [4]:
weights_hidden = np.array([
    [0.5, 0.4],
    [0.9, 1.0] 
])

Weight at output layer

Bila pecah macam ni lagi senang nak detect and adjust. Still maintain at `weight_output` sebab just in case ada dua node at output

In [5]:
weights_output = np.array([
    [-1.2, 1.1],
])

Threshold values

Sepatutnya, threshold * biased kan? Why dia tak ambil biased :( - Dia guna dekat step 3 hahahahah

Each column represent each neuron at the hidden layer

Macam mana kalau tak ada threshold? kah kah

In [6]:
thresholds_hidden = np.array([0.8, -0.1])

In [7]:
biaseds_hidden = np.array([-1, -1])

In [8]:
thresholds_output = np.array([0.3])

In [9]:
biaseds_output = np.array([-1])

Assuming dekat output layer boleh jadi more than one output

In [10]:
y_desired = np.array([0]) 

## Step 2: Activation

Activate neural network by applying inputs and desired outputs

In [11]:
def activation_function_hidden(x_inputs, weights, thresholds):
    # TODO fix docstring
    """
        Calculate actual output at hidden neurons.
        In this case, neuron 3 and neuron 4

        The actual output at hidden neuron will be input for output layer
        This function return Numpy array if possible. 

        Formula tu calculate actual output, y
        y = sigmoid(sum(xi * wij) - thres)

        where:
            xi  = input variables
            wij = weight values; 
                i = neuron at input layer, 
                j = neuron at hidden layer 
    """
    actual_output = []
    x_inputs_hidden = []

    # Multiply all x_input with 
    for x in range(len(weights)):
        for x_input in x_inputs:
            actual_output_input_x_hidden = x_input * weights[x]
            actual_output.append(actual_output_input_x_hidden)

    actual_outputs_sum = np.sum(
        np.array(actual_output),
        axis=1
    )

    # Multiply the value 
    for x, actual_output_sum in enumerate(actual_outputs_sum):
        # print(actual_output_sum)
        actual_outputs_sum[x] = actual_output_sum - (thresholds[x])
    
    # Sigmoid activation
    for actual_output_sum in actual_outputs_sum:
        x_inputs_hidden.append(round(1 / (1  + exp(-1 * actual_output_sum)), 4))
    
    return np.array([x_inputs_hidden])

In [12]:
x_inputs_output = activation_function_hidden(x_inputs=x_inputs, weights=weights_hidden, thresholds=thresholds_hidden)

In [30]:
x_inputs_output

array([[0.525 , 0.8808]])

In [13]:
y_actual = activation_function_hidden(x_inputs=x_inputs_output, weights=weights_output, thresholds=thresholds_output)

Error in numpy array lol

In [27]:
y_actual

array([[0.5097]])

In [14]:
errors = y_desired - y_actual

In [15]:
print(errors)

[[-0.5097]]


## Step 3: Weight Training

Calculate error gradient at output layer

Step 3.1: Weight and Error (Delta Error and Error Gradient) Calculation

In [16]:
def calc_err_gradient(y_outputs, errors, weights=[]):
    err_gradient = np.array([])
    if not len(weights) == 0:
        for weight in weights:
            for error in errors:
                for y_output in (y_outputs):
                    for x in range(len(y_output)):
                        err_gradient = np.append(err_gradient, values=round(y_output[x] * (1 - y_output[x]) * error * weight[x], 4))
    else:
        for error in errors:
            for y_output in y_outputs:
                for x in range(len(y_output)):
                    err_gradient = np.append(err_gradient, values=round(y_output[x] * (1 - y_output[x]) * error[x], 4)) 
    
    return err_gradient

In [17]:
err_gradient_output = calc_err_gradient(y_outputs=y_actual, errors=errors)

In [28]:
err_gradient_output

array([-0.1274])

In [18]:
err_gradient_hidden = calc_err_gradient(y_outputs=x_inputs_output, errors=err_gradient_output, weights=weights_output)

In [29]:
err_gradient_hidden

array([ 0.0381, -0.0147])

In [19]:
print(err_gradient_hidden)

[ 0.0381 -0.0147]


In [20]:
def calc_delta_weight(x_inputs, err_gradients, biaseds):
    delta_weight = np.array([])
    delta_threshold = np.array([])
    temp = []
    neurons = []

    for x in range(len(err_gradients)):
        temp.clear()
        for x_input in x_inputs:  
            for y in range(len(x_input)):
                temp.append(round(learning_rate * x_input[y] * err_gradients[x], 4))
            neurons.append(temp.copy())
    
    delta_weight = np.array(neurons)
    print(delta_weight)

    for x, biased in enumerate(biaseds):
            delta_threshold = np.append(delta_threshold, round(learning_rate * biased * err_gradients[x], 4))
        
    return {
        'delta_weight': delta_weight,
        'delta_threshold': delta_threshold
    }

In [21]:
delta_output = calc_delta_weight(x_inputs=x_inputs_output, err_gradients=err_gradient_output, biaseds=biaseds_output)

[[-0.0067 -0.0112]]


In [22]:
delta_hidden = calc_delta_weight(x_inputs=x_inputs, err_gradients=err_gradient_hidden, biaseds=biaseds_hidden)

[[ 0.0038  0.0038]
 [-0.0015 -0.0015]]


Step 3.2: Weight updation

In [23]:
weights_hidden = weights_hidden + delta_hidden['delta_weight']

In [24]:
weights_output = weights_output + delta_output['delta_weight']

In [25]:
thresholds_hidden = thresholds_hidden + delta_hidden['delta_threshold']

In [26]:
thresholds_output = thresholds_output + delta_output['delta_threshold']

## Step 4: Iteration

Iterate until sum of square error is less than 0.001

In [30]:
x_one   = bpnn.BPNN(np.array([[0,0]]), np.array([0]))
x_two   = bpnn.BPNN(np.array([[1,0]]), np.array([1]))
x_three = bpnn.BPNN(np.array([[1,1]]), np.array([1]))
x_four  = bpnn.BPNN(np.array([[1,1]]), np.array([0]))

In [ ]:
bpnn.BPNN.w

In [ ]:
bpnn.BPNN.